In [25]:
import pandas as pd     # 数据表
import numpy as np     # 数组
import re     # 正则表达式
import jieba     # 中文分词
import matplotlib.pyplot as plt     # 画图
from gensim import corpora, models
import pyLDAvis     # 交互式LDA可视化
import pyLDAvis.gensim_models as gensimvis

In [30]:
df = pd.read_excel('text_analysis_weibo_sample.xlsx', index_col = 0)

In [31]:
df.head()

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东


### 语料预处理 

#### 剔除符号与数字

In [5]:
def remove_nums(text):
    nonums = re.sub('[^\u4e00-\u9fa5]+', '', text)
    return nonums
test = df['标题/微博内容'][0]
remove_nums(test)

'高校通报教师图书馆打电话声音过大出言不逊公道自在人心谣言自在人心'

#### 分词

In [8]:
# 加载中文停用词词典，可个性化设置
stopwords = open('stopwords.txt', encoding = 'utf-8').read()

def clean_text(text):
    words = jieba.lcut(text)
    words = [w for w in words if w not in stopwords and w!='[' and w!=']']
    return ' '.join(words)
test = df['标题/微博内容'][0]
clean_text(test)

'高校 通报 教师 图书馆 打电话 声音 过大 出言不逊 公道 人心 谣言 人心 \u200b \u200b'

In [36]:
df['标题/微博内容'] = df['标题/微博内容'].astype(str)
df['微博内容分词'] = df['标题/微博内容'].apply(remove_nums)
df['微博内容分词'] = df['微博内容分词'].apply(clean_text)
df['微博内容分词'] = df['微博内容分词'].apply(lambda x: x.split())
df

,index,标题/微博内容,点赞,转发,评论,账号昵称UID加密,粉丝数,关注数,地域,微博内容分词
0,34121,国债：地产行业重磅利好提振风险偏好，期债低开低走 国债期货全线收跌，10年期主力...,0,0,0,e5df796860e68f403bcf9651bab4d42e,0,0,其他,"[国债, 地产, 行业, 重磅, 利好, 提振, 风险, 偏好, 期债, 低开, 低, 走,..."
1,40230,#喜迎二十大 忠诚保平安#,0,0,0,6e35cb69ad52f20de5e28197b2e85306,405444,252,广西,"[喜迎, 二十大, 忠诚, 保平安]"
2,7714,注意！事关明日教资考试！福建省教育考试院发布补充公告 福建省2022年下半年全国中小学教师...,0,0,0,e6953217442e6c06a7af23eee5e185f2,53264,2177,福建,"[事关, 明日, 教资, 考试, 福建省, 教育, 考试院, 发布, 补充, 公告, 福建省..."
3,27378,近日，“千年大计”雄安新区迎来五周岁生日。从“一张白纸...,0,0,0,NaN,0,0,北京,"[近日, 千年, 大计, 雄安, 新区, 迎来, 五周岁, 生日, 一张白纸, 塔吊, 林立..."
4,15435,樊振东牛逼！,0,0,0,344af41eac516375c04dee6325e763cc,8,51,山东,"[樊振东, 牛, 逼]"
...,...,...,...,...,...,...,...,...,...,...
95,14034,老公好漂亮[舔屏],0,0,0,aa7af97ae98cbe8983e559b948ecfabf,825,191,北京,"[老公, 漂亮, 舔, 屏]"
96,35249,恭喜@张小娜呀娜 1名用户获得【小飞T】。C官方唯一抽奖工具@C抽奖平台 对本次抽奖进行监督...,0,0,2,f538513e5801c275cbcf285517a8ee62,51709,20,北京,"[恭喜, 张小娜, 娜, 名, 用户, 小飞, 官方, 唯一, 抽奖, 工具, 抽奖, 平台..."
97,39598,接不动了，居民部门负债率提升空间太有限了。现在动辄几万一平米，总价高，月供高，预期转差，还怎...,0,0,0,2e76c39b6665a0b6f5bdc4c35f36f573,0,59,四川,"[接不动, 居民, 部门, 负债率, 提升, 空间, 有限, 动辄, 几万, 平米, 总价,..."
98,48977,目前电子烟政策已从制定阶段逐渐走向实施阶段，而主要的电子烟公司股价下跌幅度高达70%-...,0,0,0,NaN,0,0,北京,"[电子, 烟, 政策, 制定, 阶段, 走向, 实施, 阶段, 电子, 烟, 股价, 下跌,..."


### LDA

In [37]:
dictionary = corpora.Dictionary(df['微博内容分词'])     # 根据分词结果创建字典
corpus = [dictionary.doc2bow(text) for text in df['微博内容分词']]     # 根据分词结果创建语料库

In [38]:
# 训练LDA模型
lda_model = models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)

In [39]:
# 查看主题
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.015*"郑州" + 0.013*"亿元" + 0.011*"重庆" + 0.010*"城市" + 0.007*"增长"')
(1, '0.009*"呜呜" + 0.004*"死" + 0.004*"笑" + 0.004*"哈哈哈哈" + 0.004*"坠机"')
(2, '0.009*"刘宇" + 0.005*"冬奥" + 0.004*"雪舞" + 0.004*"烊" + 0.004*"话筒"')
(3, '0.027*"中国" + 0.021*"经济" + 0.009*"增长" + 0.009*"发展" + 0.009*"世界"')
(4, '0.008*"月" + 0.008*"市场" + 0.007*"机构" + 0.006*"资产" + 0.005*"底部"')
(5, '0.030*"电子" + 0.030*"烟" + 0.011*"政策" + 0.010*"月" + 0.010*"市场"')
(6, '0.012*"开户" + 0.011*"期货" + 0.009*"刘雨昕" + 0.009*"玉米" + 0.008*"产品"')
(7, '0.018*"数字" + 0.012*"网络" + 0.012*"建设" + 0.010*"发展" + 0.009*"基础设施"')
(8, '0.031*"考生" + 0.018*"考试" + 0.016*"考场" + 0.011*"考点" + 0.009*"核酸"')
(9, '0.021*"月" + 0.018*"万吨" + 0.013*"市场" + 0.011*"日" + 0.011*"库存"')


### 可视化

In [41]:
lda_vis = gensimvis.prepare(lda_model, corpus, dictionary, n_jobs=1)
# 备注：上述语句如果在数据量比较大的时候跑不出来，可以选择加一个n_jobs=1的参数，降低计算量，避免报错
# 显示可视化界面
pyLDAvis.display(lda_vis)

In [42]:
# 导出可视化结果到html
pyLDAvis.save_html(lda_vis, 'lda_visualization.html')